## Introduction

This notebook will introduce some properties of **transformation matrices** as used in the [**Cairo**](http://cairographics.org/) 2D graphics library. Access to Cairo in Python will be done via the [**Qahirah**](https://github.com/ldo/qahirah) high-level binding.

The following topics will be covered:
* [What Is A Matrix?](#What-Is-The-Matrix?)
* [Common Types Of Matrices](#Common-Types-Of-Matrices)
* [Angles: Radians Or Degrees?](#Angles:-Radians-Or-Degrees?)
* [Inverse Transformations](#Inverse-Transformations)
* [Combining Transformations](#Combining-Transformations)
* [When Do Transformations Commute?](#When-Do-Transformations-Commute?)
* [Transforming About An Arbitrary Point](#Transforming-About-An-Arbitrary-Point)
* [Transforming Along An Arbitrary Orientation](#Transforming-Along-An-Arbitrary-Orientation)
* [Portrait-Landscape Orientation](#Portrait/Landscape-Orientation)
* [Inverting The *Y*-Axis](#Inverting-The-Y-Axis)

First, set up some common definitions which will be reused later.

In [ ]:
from ipywidgets.widgets import \
    interact
import ipywidgets.widgets as \
    widgets
from IPython.display import \
    display_png, Latex
from IPython.display import \
    display as ipython_display
import qahirah as qah
from qahirah import \
    CAIRO, \
    Colour, \
    Matrix, \
    Path, \
    Rect, \
    Vector

pix = qah.ImageSurface.create \
  (
    format = CAIRO.FORMAT_RGB24,
    dimensions = (400, 400)
  )
ctx = None

def reset() :
    "(re)initializes the drawing context, wiping out any existing drawing."
    global ctx
    del ctx
    ctx = qah.Context.create(pix)
    (ctx
       .save()
       .set_source_colour(Colour.grey(.95))
       .paint()
       .restore()
    )
#end reset

def display() :
    "(re)displays what has been drawn."
    display_png(pix.to_png_bytes(), raw = True)
#end display

def format_matrix(m) :
    "generates a Latex markup representation of a Matrix."
    return \
          (
            r"\begin{{bmatrix}}"
            r"{:.7g} & {:.7g} & {:.7g} \\"
            r"{:.7g} & {:.7g} & {:.7g} \\"
            r"0 & 0 & 1"
            r"\end{{bmatrix}}"
            .format(m.xx, m.xy, m.x0, m.yx, m.yy, m.y0)
          )
#end format_matrix

def format_vector_row(v) :
    "generates a Latex markup representation of a Vector in row form."
    return \
        "$({:.7g}, {:.7g})$".format(v.x, v.y)
#end format_vector_row

def format_vector_col(v) :
    "generates a Latex markup representation of a Vector in column form."
    return \
          (
            r"\begin{{bmatrix}}"
            r"{:.7g} \\"
            r"{:.7g} \\"
            r"1"
            r"\end{{bmatrix}}"
            .format(v.x, v.y)
          )
#end format_vector_col

def display_latex(s) :
    "formats and displays a string as Latex markup."
    ipython_display(Latex(s))
#end display_latex

reset()

def draw_marker(pos : Vector, label : str, colour : Colour) :
    "draws a marker into ctx at pos and gives it a label."
    pos = Vector.from_tuple(pos)
    marker_size = 8
    marker_radius = 6
    marker_gap = 2
    (ctx
         .save()
         .set_line_width(1)
         .new_path()
         .arc
           (
              centre = pos,
              radius = marker_radius,
              angle1 = 0,
              angle2 = qah.circle,
              negative = False
           )
         .set_source_colour(colour)
         .fill_preserve()
         .set_source_colour(Colour.grey(0))
         .stroke()
    )
    for i in range(4) :
        angle = i / 4 * qah.circle
        (ctx
             .move_to(pos + Vector(marker_gap, 0).rotate(angle))
             .line_to(pos + Vector(marker_size, 0).rotate(angle))
        )
    #end for
    ctx.stroke()
    if label != None :
        (ctx
             .move_to(pos + Vector(5, 5))
             .set_source_colour(Colour.grey(0))
             .show_text(label)
             .restore()
        )
    #end if
#end draw_marker

def draw_axes() :
    radius = pix.dimensions / 2
    ctx.matrix = Matrix.translate(radius)
    (ctx
         .save()
         .set_source_colour(Colour.from_hsva((0.5, 1, 1)))
         .set_line_width(1)
         .new_path()
         .move_to(Vector(1, 0) * radius)
         .line_to(Vector(-1, 0) * radius)
         .move_to(Vector(0, 1) * radius)
         .line_to(Vector(0, -1) * radius)
         .stroke()
         .restore()
    )
#end draw_axes

def display_points(points) :
    reset()
    draw_axes()
    for point in points :
        draw_marker(point, None, Colour.grey(0.75))
    #end for
    display()
#end display_points

marker_1_colour = Colour.from_hsva((0.5, 0.5, 0.5))
marker_2_colour = Colour.from_hsva((0.8, 0.5, 0.5))

## What Is The Matrix?

A matrix defines a **linear transformation** of coordinate space. What makes the transformation “linear” is that lines which were straight before the transformation remain straight after the transformation.

Actually, Cairo only supports a *subset* of linear transformations: these are the **affine transformations**, which are further restricted in that straight lines which were parallel before the transformation remain parallel afterwards. The linear transformations that are excluded from this set are the *perspective distortions*.

Cairo provides for directly constructing transformations of the following kinds:

* *translation* — a change of position.
* *scaling* — a change of size. If the scale factors along the $x$- and $y$-axes are different, then this is a *nonuniform* scaling; if they are the same, the scaling is *uniform*.
* *rotation* — a change of orientation.

There is an additional kind of transformation that does work in Cairo, though oddly there is no convenience routine for directly constructing it:

* *skew* or *shear* — a distortion of shapes that turns rectangles into parallelograms.

The Qahirah binding does provide a routine for making these.

Translations and rotations make up the *rigid-body* transformations; the distance between any two points in an object or scene does not change after they go through any combination of translations and rotations.

Multiple linear transformations may be freely combined. Transformations are usually expressed in *homogeneous coordinates*, because it means that all the possible transformations can be equally easily expressed in terms of matrix multiplications.

In homogeneous coordinates, a 2D vector actually has 3 components, while transformation matrices have 3 × 3 = 9 components. Transformation of a homogenous vector $(x, y, w)$ to $(x', y', w')$ can be expressed as *premultiplication of a column vector by the transformation matrix*, thus:

$$\begin{bmatrix}
x' \\
y' \\
w'
\end{bmatrix}
=
\begin{bmatrix}
m_{11} & m_{12} & m_{13} \\
m_{21} & m_{22} & m_{23} \\
m_{31} & m_{32} & m_{33}
\end{bmatrix}
\cdotp
\begin{bmatrix}
x \\
y \\
w
\end{bmatrix}
=
\begin{bmatrix}
m_{11}x + m_{12}y + m_{13}w \\
m_{21}x + m_{22}y + m_{23}w \\
m_{31}x + m_{32}y + m_{33}w
\end{bmatrix}
$$

It could also be expressed as postmultiplication of a row vector, but the [premultiplication convention is preferred](http://cairographics.org/cookbook/matrix_conventions/).

Because Cairo only allows affine transformations, this constrains the values of some of the vector and matrix components above: the homogeneous coordinate $w$ and $w'$ must always be one, and the bottommost row of the matrix is similarly constrained to constant values. Thus, the general form of a Cairo transformation is only:

$$\begin{bmatrix}
x' \\
y' \\
1
\end{bmatrix}
=
\begin{bmatrix}
m_{11} & m_{12} & m_{13} \\
m_{21} & m_{22} & m_{23} \\
0 & 0 & 1
\end{bmatrix}
\cdotp
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix}
=
\begin{bmatrix}
m_{11}x + m_{12}y + m_{13} \\
m_{21}x + m_{22}y + m_{23} \\
1
\end{bmatrix}
$$

Hence, the [<tt>cairo_matrix_t</tt>](http://cairographics.org/manual/cairo-cairo-matrix-t.html) structure only needs to hold 6 elements, not 9.

## Common Types Of Matrices

It is handy to be able to recognize certain common simple forms of transformation matrix. First of all, the *identity* matrix:

$$
I =
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

This transforms a vector to itself, *i.e.* leaves it unchanged:

$$\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
\cdotp
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix}
=
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix}
$$

A translation matrix, which moves by $t_x$ along the $x$-axis and $t_y$ along the $y$-axis:
$$\begin{bmatrix}
1 & 0 & t_x \\
0 & 1 & t_y \\
0 & 0 & 1
\end{bmatrix}
\cdotp
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix}
=
\begin{bmatrix}
x + t_x \\
y + t_y \\
1
\end{bmatrix}
$$

Translation example:

In [ ]:
def transformation_common(mat) :
    example_points = [Vector(50, -50), Vector(25, 50)]
    for point in example_points :
        display_latex \
          (
            "$${} \\cdotp {} = {}$$"
            .format
              (
                format_matrix(mat),
                format_vector_col(point),
                format_vector_col(mat * point),
              )
          )
    #end for
    display_points(mat.mapiter(example_points))
#end transformation_common

@interact(tx = (-100.0, +100.0, 10.0), ty = (-100.0, +100.0, 10.0))
def translation_example(tx, ty) :
    transformation_common(Matrix.translate((tx, ty)))
#end translation_example

Scaling about the origin, by $s_x$ along the $x$-axis and $s_y$ along the $y$-axis (the scaling is *uniform* iff $s_x = s_y$):
$$\begin{bmatrix}
s_x & 0 & 0 \\
0 & s_y & 0 \\
0 & 0 & 1
\end{bmatrix}
\cdotp
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix}
=
\begin{bmatrix}
s_x x \\
s_y y \\
1
\end{bmatrix}
$$

Scaling example:

In [ ]:
@interact(sx = (-1.0, 2.0, 0.5), sy = (-1.0, 2.0, 0.5))
def scaling_example(sx, sy) :
    transformation_common(Matrix.scale((sx, sy)))
#end scaling_example

Rotation by an angle $\theta$ about the origin:
$$\begin{bmatrix}
\cos \theta & - \sin \theta & 0 \\
\sin \theta & \cos \theta & 0 \\
0 & 0 & 1
\end{bmatrix}
\cdotp
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix}
=
\begin{bmatrix}
x \cos \theta - y \sin \theta \\
x \sin \theta + y \cos \theta \\
1
\end{bmatrix}
$$

Rotation example:

In [ ]:
@interact(degrees = (-90.0, 90.0, 10.0))
def rotation_example(degrees) :
    transformation_common(Matrix.rotate(degrees * qah.deg))
#end rotation_example

## Angles: Radians Or Degrees?

One vexing question when designing a graphics API is whether to specify angles in degrees or radians. Whereas humans may often prefer to work with angles in degrees, there is one inescapable point:

> **Trigonometric calculations are almost always easier in radians.**

Therefore, the simplest design is only to worry about radians/degrees conversions when accepting input from the user, and when displaying output to the user. **At all points in-between, angles should be represented in radians.** This is how Cairo does it, and it is how Qahirah does it.

For convenience, Qahirah provides the <tt>deg</tt> constant, equal to $\pi \over 180$: multiplying a number of degrees by this converts it to radians, and correspondingly dividing a number of radians by this converts it to degrees.

Qahirah also provides the <tt>circle</tt> constant, equal to $2 \pi$: dividing radians by this converts it to units of whole circles, and multiplying such units by this converts them to radians.

Thus, <tt>circle</tt> (or <tt>1 \* circle</tt>) is equivalent to <tt>360 \* deg</tt> which is 360°, <tt>0.5 \* circle</tt> to <tt>180 \* deg</tt> or 180°, <tt>0.25 \* circle</tt> to <tt>90 \* deg</tt> or 90°, and so on.

## Inverse Transformations

The *inverse* of a transformation is that which transforms the transformed vector $P'$ back to the original vector $P$. The inverse of the transformation $M$ as in

$$P' = M \cdotp P$$

is written $M^{-1}$ as in

$$P = M^{-1} \cdotp P'$$

The inverse of the inverse is, of course, the original matrix:

$$(M^{-1})^{-1} = M$$

The identity matrix is the inverse of itself:

$$I^{-1} = I$$

The inverse of an affine transformation is an affine transformation. The above simple transformation cases have equally simple inverses.

Inverse translation:
$$\begin{bmatrix}
0 & 0 & t_x \\
0 & 0 & t_y \\
0 & 0 & 1
\end{bmatrix}
^{-1} =
\begin{bmatrix}
0 & 0 & - t_x \\
0 & 0 & - t_y \\
0 & 0 & 1
\end{bmatrix}
$$

Inverse translation example:

In [ ]:
def inverse_transformation_common(mat : Matrix) :
    "common routine for demonstrating inverse transformations."
    reset()
    draw_axes()
    inv_mat = mat.inv()
    point = Vector(50, -50)
    display_latex \
      (
        "$${} \\cdotp {} = {}\\\\\n"
        "{} \\cdotp {} = {}\\\\\n"
        "{} \\cdotp {} = {}$$"
        .format
          (
            format_matrix(mat),
            format_vector_col(point),
            format_vector_col(mat * point),
            format_matrix(inv_mat),
            format_vector_col(point),
            format_vector_col(inv_mat * point),
            format_matrix(mat),
            format_matrix(inv_mat),
            format_matrix(mat * inv_mat),
          )
      )
    draw_marker(mat * point, None, marker_1_colour)
    draw_marker(inv_mat * point, None, marker_2_colour)
    display()
#end inverse_transformation_common

@interact(tx = (-100.0, +100.0, 10.0), ty = (-100.0, +100.0, 10.0))
def inverse_translation_example(tx, ty) :
    inverse_transformation_common(Matrix.translate((tx, ty)))
#end inverse_translation_example

Inverse scaling about the origin:
$$\begin{bmatrix}
s_x & 0 & 0 \\
0 & s_y & 0 \\
0 & 0 & 1
\end{bmatrix}
^{-1} =
\begin{bmatrix}
1 \over s_x & 0 & 0 \\
0 & 1 \over s_y & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

Inverse scaling example:

In [ ]:
@interact(sx = (0.5, 3.0, 0.5), sy = (0.5, 3.0, 0.5))
def inverse_scaling_example(sx, sy) :
    inverse_transformation_common(Matrix.scale((sx, sy)))
#end inverse_scaling_example

Inverse rotation about the origin (or rotation by the angle $- \theta$):

$$\begin{bmatrix}
\cos \theta & - \sin \theta & 0 \\
\sin \theta & \cos \theta & 0 \\
0 & 0 & 1
\end{bmatrix}
^{-1} =
\begin{bmatrix}
\cos \theta & \sin \theta & 0 \\
- \sin \theta & \cos \theta & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

Inverse rotation example:

In [ ]:
@interact(degrees = (-90.0, 90.0, 10.0))
def inverse_rotation_example(degrees) :
    inverse_transformation_common(Matrix.rotate(degrees * qah.deg))
#end inverse_rotation_example

More complex matrices will have more complex inverses. **Not all matrices have inverses**: those which do not are called *degenerate*, because they represent transformations that collapse all of 2D space onto an infinitely thin line, or even a single point.

Qahirah provides the <tt>det()</tt> method for computing the *determinant* of a <tt>Matrix</tt> object. Degenerate matrices will be those that have a determinant of zero; if the determinant is nonzero, the matrix will have an inverse.

## Combining Transformations

As hinted at already, more elaborate transformations may be built up by combining simpler ones. Two matrices

$$M =
\begin{bmatrix}
m_{11} & m_{12} & m_{13} \\
m_{21} & m_{22} & m_{23} \\
m_{31} & m_{32} & m_{33}
\end{bmatrix}
$$

and

$$N =
\begin{bmatrix}
n_{11} & n_{12} & n_{13} \\
n_{21} & n_{22} & n_{23} \\
n_{31} & n_{32} & n_{33}
\end{bmatrix}
$$

may be combined by matrix multiplication:

$$M \cdotp N =
\begin{bmatrix}
m_{11} & m_{12} & m_{13} \\
m_{21} & m_{22} & m_{23} \\
m_{31} & m_{32} & m_{33}
\end{bmatrix}
\cdotp
\begin{bmatrix}
n_{11} & n_{12} & n_{13} \\
n_{21} & n_{22} & n_{23} \\
n_{31} & n_{32} & n_{33}
\end{bmatrix}
=
\begin{bmatrix}
m_{11} n_{11} + m_{12} n_{21} + m_{13} n_{31} & m_{11} n_{12} + m_{12} n_{22} + m_{13} n_{32} & m_{11} n_{13} + m_{12} n_{23} + m_{13} n_{33}  \\
m_{21} n_{11} + m_{22} n_{21} + m_{23} n_{31} & m_{21} n_{12} + m_{22} n_{22} + m_{23} n_{32} & m_{21} n_{13} + m_{22} n_{23} + m_{23} n_{33} \\
m_{31} n_{11} + m_{32} n_{21} + m_{33} n_{31} & m_{31} n_{12} + m_{32} n_{22} + m_{33} n_{32} & m_{31} n_{13} + m_{32} n_{23} + m_{33} n_{33}
\end{bmatrix}
$$

Or, restricting to affine transformations only:

$$
\begin{bmatrix}
m_{11} & m_{12} & m_{13} \\
m_{21} & m_{22} & m_{23} \\
0 & 0 & 1
\end{bmatrix}
\cdotp
\begin{bmatrix}
n_{11} & n_{12} & n_{13} \\
n_{21} & n_{22} & n_{23} \\
0 & 0 & 1
\end{bmatrix}
=
\begin{bmatrix}
m_{11} n_{11} + m_{12} n_{21} & m_{11} n_{12} + m_{12} n_{22} & m_{11} n_{13} + m_{12} n_{23} + m_{13}  \\
m_{21} n_{11} + m_{22} n_{21} & m_{21} n_{12} + m_{22} n_{22} & m_{21} n_{13} + m_{22} n_{23} + m_{23} \\
0 & 0 & 1
\end{bmatrix}
$$

Note that matrix multiplication is *associative*; if $L$ is also a matrix:

$$(L \cdotp M) \cdotp N = L \cdotp (M \cdotp N)$$

but it is not *commutative*: in general,

$$M \cdotp N \neq N \cdotp M$$

An exception is when $N$ is the inverse of $M$ (and vice versa). Multiplication of a matrix by its inverse, in either order, gives the identity matrix:

$$M^{-1} \cdotp M = M \cdotp M^{-1} = I$$

Also commutative is multiplication by the identity matrix:

$$M \cdotp I = I \cdotp M = M$$

One interesting property of inverse transformations is that the *inverse of a combination of transformations is the combination of their inverses in reverse order*. That is,

$$(M \cdotp N)^{-1} = N^{-1} \cdotp M^{-1}$$

The following example demonstrates the combination of a translation and rotation transformation, applied in either order to the same pair of points, which each order shown in a different colour.

In [ ]:
def combined_transformation_common(mat1 : Matrix, mat2 : Matrix) :
    example_points = [Vector(50, -50), Vector(25, 50)]
    reset()
    draw_axes()
    for point in example_points :
        draw_marker(mat2 * mat1 * point, None, marker_1_colour)
        draw_marker(mat1 * mat2 * point, None, marker_2_colour)
    #end for
    display()
#end combined_transformation_common

@interact(tx = (-100.0, +100.0, 10.0), ty = (-100.0, +100.0, 10.0), degrees = (-90.0, 90.0, 10.0))
def translation_rotation_example(tx, ty, degrees) :
    combined_transformation_common(Matrix.translate((tx, ty)), Matrix.rotate(degrees * qah.deg))
#end translation_rotation_example

## When Do Transformations Commute?

As mentioned above, transformations cannot be freely reordered in the general case. Besides the previously-noted exceptions, the following are some additional special cases where transformations *do* commute:

* A sequence of translations
* A sequence of scalings around the same origin
* A sequence of rotations around the same axis
* A sequence of rotations about the same axis and *uniform* scalings about the same point on that axis

In each of these cases, the transformations in the sequence may be freely reordered, and the effect will be the same (barring rounding errors).

## Transforming About An Arbitrary Point

Cairo’s [scaling and rotation calls](http://cairographics.org/manual/cairo-Transformations.html) (and the [matrix ones](http://cairographics.org/manual/cairo-cairo-matrix-t.html) as well) apply their transformations about the origin. Qahirah augments its equivalents of these calls to allow specification of an arbitrary origin. But what if it didn’t—how would you implement transformation about an arbitrary origin?

The answer is quite simple: decompose the desired operation into a sequence of simpler transformations as follows:
* translate the desired point to the origin
* perform the desired transformation about the origin
* translate the origin back to the original point.

Because of the premultiplication convention, these separate transformations have to be written in reverse order. If you look at the source code for Qahirah’s `Matrix.scale`, `Matrix.rotate` and `Matrix.skew` methods, you will see that they implement exactly the above procedure.

## Transforming Along An Arbitrary Orientation

The scaling and skew transformations allow for specifying separate components along the $x$- and $y$-axes. But what if you wanted to define scaling or skewing along some other direction? You could directly work out the components of the operation along the $x$- and $y$-axes, which will take a bit of maths calculation. But another, easier, way is to use an analogue to the previous procedure for generalizing an origin-centric transformation to one about an arbitrary point, by combining the following sequence of simpler transformations:

* Rotate the desired direction to be aligned to, say, the $x$-axis
* Apply the desired scaling or skewing along the $x$-axis
* Rotate the $x$-axis back to the original direction.

You could also use the $y$-axis instead of the $x$-axis; the procedure would work just as well.

The following code demonstrates scaling a pair of example points along the direction of the coloured line:

In [ ]:
def oriented_transform(xform : Matrix, orient : float) :
    "returns xform adjusted to apply along the specified orient in degrees."
    return \
        Matrix.rotate(orient) * xform * Matrix.rotate(- orient)
#end oriented_transform

@interact(scale = (0.5, 5.0, 0.5), orient = (-90, 90, 10))
def oriented_scale_demo(scale, orient) :
    example_points = [Vector(30, 10), - Vector(30, 10)]
    orient *= qah.deg
    reset()
    draw_axes()
    xform = oriented_transform(Matrix.scale(Vector(scale, 1)), orient)
    for point in example_points :
        draw_marker(xform * point, None, marker_1_colour)
    #end for
    radius = pix.dimensions / 2
    (ctx
         .save()
         .rotate(orient)
         .move_to(Vector(-1, 0) * radius)
         .line_to(Vector(1, 0) * radius)
         .set_line_width(1)
         .set_source_colour(marker_2_colour)
         .stroke()
         .restore()
     )
    display()
#end oriented_scale_demo

**Exercise:** What if you wanted to perform some transformation about an arbitrary point *and* along some arbitrary orientation? How would you combine *both* the above techniques?

## Portrait/Landscape Orientation

Quite often you have a need to render a page in a rotated orientation, commonly described as “portrait” versus “landscape” orientation. In fact, there are four different orientations in which one may render to a rectangular page, while keeping things aligned with the edges of the page.

Not only must you *rotate* the coordinate system by some suitable multiple of 90° for each orientation, you must also *translate* the origin of the coordinate system to the corresponding corner of the page. The direction of the translation depends not just on the orientation, but on the order in which you apply the transformations: in other words, on whether you apply the translation before or after the rotation.

The following demo illustrates the correct calculations for all the cases:

In [ ]:
orientations = \
    {
        "0°" :
            {
                "rotate" : 0,
                "pre_translate" : Vector(0, 0),
                "post_translate" : Vector(0, 0),
            },
        "90°" :
            {
                "rotate" : 90 * qah.deg,
                "pre_translate" : Vector(0, -1),
                "post_translate" : Vector(1, 0),
            },
        "180°" :
            {
                "rotate" : 180 * qah.deg,
                "pre_translate" : Vector(-1, -1),
                "post_translate" : Vector(1, 1),
            },
        "270°" :
            {
                "rotate" : 270 * qah.deg,
                "pre_translate" : Vector(-1, 0),
                "post_translate" : Vector(0, 1),
            },
    }

@interact \
  (
    orientation =
        widgets.Dropdown
          (
            options = list
              (
                (o, o) for o in orientations
              ),
            value = "0°"
          ),
    translate_order =
          widgets.Dropdown
            (
              options =
                [
                    ("Pre-Translate", False),
                    ("Post-Translate", True),
                ],
              value = True
            )
  )
def orientation_demo(orientation, translate_order) :
    orient = orientations[orientation]
    reset()
    if translate_order :
        ctx.transform \
          (
                Matrix.translate(orient["post_translate"] * pix.dimensions)
            *
                Matrix.rotate(orient["rotate"])
          )
    else :
        ctx.transform \
          (
                Matrix.rotate(orient["rotate"])
            *
                Matrix.translate(orient["pre_translate"] * pix.dimensions)
          )
    #end if
    (ctx
         .move_to((0, 0))
         .line_to(Vector(1, 0) * pix.dimensions)
         .move_to((0, 0))
         .line_to(Vector(0, 1) * pix.dimensions)
         .set_line_width(8)
         .stroke()
         .move_to(Vector(0.2, 0.2) * pix.dimensions)
         .set_font_size(24)
         .show_text("Orientation")
    )
    display()
#end orientation_demo

## Inverting The *Y*-Axis

There is often a need to adapt graphics or drawing code written for some other API, such as PostScript or OpenGL, where the *y*-axis normally increases *up* the vertical, rather than *down* it as is usual in Cairo. Rather than go through and change all the numbers, it is easier to simply apply a corrective inverting transformation before rendering the graphics.

Constructing this corrective transformation is easy: simply apply a scaling by a factor of $-1$ along the *y*-axis, around the line $y = {h \over 2}$, where $h$ is the height of the image area.

However, note that if drawing includes display of any text, this will end up upside-down. To fix this, a separate inverting transformation needs to be applied to the font matrix.

In [ ]:
@interact \
  (
    invert_coords = widgets.Checkbox(value = False),
    invert_font_matrix = widgets.Checkbox(value = False),
  )
def flip_coords_demo(invert_coords, invert_font_matrix) :
    reset()
    if invert_coords :
        ctx.transform(Matrix.scale(factor = Vector(1, -1), centre = pix.dimensions / 2))
    #end if
    ctx.set_font_size(24)
    if invert_font_matrix :
        ctx.font_matrix *= Matrix.scale((1, -1))
    #end if
    ctx.move_to((100, 100))
    ctx.show_text("Orientation")
    display()
#end flip_coords_demo